In [7]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install langgraph

In [8]:
from google.colab import userdata
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["ALPHAVANTAGE_API_KEY"] = userdata.get('ALPHAVANTAGE_API_KEY')

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


model = ChatOpenAI(model="gpt-4o-mini")
dummy_msg = HumanMessage(content="Hi, my name is Ana")

model.invoke([dummy_msg])

AIMessage(content='Hi Ana! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19', 'finish_reason': 'stop', 'logprobs': None}, id='run-62cf86e7-19a3-4593-82d3-32e1c1791ac7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23})

In [24]:
# Defining the tool Alpha Vantage wrapper

from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool

fin_tool = AlphaVantageAPIWrapper()

fin_tool._get_exchange_rate("EUR", "IDR")

{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'EUR',
  '2. From_Currency Name': 'Euro',
  '3. To_Currency Code': 'IDR',
  '4. To_Currency Name': 'Indonesian Rupiah',
  '5. Exchange Rate': '17736.00000000',
  '6. Last Refreshed': '2024-08-05 12:56:10',
  '7. Time Zone': 'UTC',
  '8. Bid Price': '17735.30000000',
  '9. Ask Price': '17736.50000000'}}

In [19]:
@tool
def divisa_change(amount: float, from_currency: str, to_currency: str) -> int:
    """Currency change."""
    response = fin_tool._get_exchange_rate(from_currency, to_currency)
    return  amount * float(response["Realtime Currency Exchange Rate"]['5. Exchange Rate'])

tools = [divisa_change]

In [25]:
# Buiding the Agent AI

from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [36]:
prompt_dict = {"messages": [HumanMessage(content= "Please convert from IDR to EUR the amount of 200k EUR")]}
response = agent_executor.invoke(prompt_dict)
print(response)


{'messages': [HumanMessage(content='Please convert from IDR to EUR the amount of 200k EUR', id='99cba18f-bdff-4996-b96d-77b0d14f5b2d'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gmIzXcFmA7ii17E2sJW4nduW', 'function': {'arguments': '{"amount":200000,"from_currency":"EUR","to_currency":"IDR"}', 'name': 'divisa_change'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 65, 'total_tokens': 92}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-857fa133-d35e-4f35-af4d-a319e147858f-0', tool_calls=[{'name': 'divisa_change', 'args': {'amount': 200000, 'from_currency': 'EUR', 'to_currency': 'IDR'}, 'id': 'call_gmIzXcFmA7ii17E2sJW4nduW', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 27, 'total_tokens': 92}), ToolMessage(content='3554400000.0', name='divisa_change', id='d0cbdda7-77b5-4419-9f73-abe794671d6

In [37]:
print(response["messages"][3].content)

The amount of 200,000 EUR is equivalent to 3,554,400,000 IDR.
